# Libraries

In [2]:
import pandas as pd
import requests
import json
from typing import List, Optional
from pydantic import BaseModel, Field, ValidationError
from glob import glob

from retry import retry
import streamlit as st
import IPython
from tqdm import tqdm
import base64

from openai import OpenAI
client = OpenAI(api_key=st.secrets["OPENAI_API_KEY"])
OPENAI_MODEL = "gpt-4o-mini"

In [21]:
df = pd.read_csv("../data/processed/chinese_sentences.csv")
df['sentences'] = df['sentences'].apply(lambda x: "".join(x.split()))
sentences_dict = df.to_dict(orient="records")

In [22]:
sentences_dict[:3]

[{'chapter': 1, 'sentences': '我姓張，是騰飛公司的業務部經理。'},
 {'chapter': 1, 'sentences': '我的名字叫張宏岳，宏是宏大的宏，岳是岳飛的岳。'},
 {'chapter': 1, 'sentences': '我叫白可凡，可是可以的可，凡是平凡的凡。我的祖籍在福建廈門，但是在廣州出生。'}]

# Steps

for each word:
- get the openai inputs
- get the google translate inputs
- get the TTS inputs


```json
[
  {
    "chapter": 1,
    "word": "謝謝大家的支持和配合",
    "pinyin": "Xièxiè dàjiā de zhīchí hé pèihé",
    "translation": "Thank you for your support and cooperation",
    "pronounciation_audio_bytes": "RIFF~vx"
  }
]
```

In [15]:
@retry(requests.exceptions.HTTPError, tries=5, delay=10)
def translate(text, from_lang, to_lang):
    # Base URL of the translation API
    url = "https://api.datpmt.com/api/v1/dictionary/translate"

    # Parameters for the GET request
    params = {
        'string': text,
        'from_lang': from_lang,
        'to_lang': to_lang
    }

    # Make the GET request to the translation API
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse JSON response
        json_response = response.json()
        return json_response # Adjust the key based on actual API response structure
    else:
        #return "Error: " + response.text
        print("Error Translation: " + response.text)
        raise requests.exceptions.HTTPError

In [16]:
@retry(requests.exceptions.HTTPError, tries=5, delay=10)
def transliterate(keyword, from_lang):
    # Base URL of the transliteration API
    url = "https://api.datpmt.com/api/v1/dictionary/transliteration"

    # Parameters for the GET request
    params = {
        'keyword': keyword,
        'from_lang': from_lang
    }

    # Make the GET request to the transliteration API
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse JSON response
        json_response = response.json()
        return json_response  # Adjust this key based on the actual API response structure
    else:
        print("Error Transliteration: " + response.text)
        raise requests.exceptions.HTTPError
        #return "Error: " + response.text

In [17]:
def text_to_speech(chinese_sentence):
    # Define the URL to your FastAPI endpoint
    url = "http://localhost:80/synthesize"

    # The data to be sent in the POST request
    data = {
        "language": "ZH",  # Replace with the language of your choice
        "speaker": "ZH",  # Replace with speaker of choice. English British accent.
        "text": chinese_sentence,
        "speed": 0.7, # must be a positive number
    }

    # Set the appropriate headers
    headers = {
        "Content-Type": "application/json"
    }
    # Make the POST request
    response = requests.post(url, data=json.dumps(data), headers=headers)
    return response.content

# Workflow

In [18]:
sentences_dict[0]

{'chapter': 1, 'sentences': '我姓張，是騰飛公司的業務部經理。'}

In [23]:
for sentence in tqdm(sentences_dict):
    print(f"processing sentence {sentence}")
    sentence["pinyin"] = transliterate(sentence["sentences"], from_lang="zh-TW")
    sentence["translation"] = translate(sentence["sentences"], from_lang="zh-TW", to_lang="en")
    sentence["pronounciation_audio_bytes"] = base64.b64encode(text_to_speech(sentence["sentences"])).decode('utf-8')

  0%|          | 0/33 [00:00<?, ?it/s]

processing sentence {'chapter': 1, 'sentences': '我姓張，是騰飛公司的業務部經理。'}


  3%|▎         | 1/33 [00:00<00:22,  1.39it/s]

processing sentence {'chapter': 1, 'sentences': '我的名字叫張宏岳，宏是宏大的宏，岳是岳飛的岳。'}


  6%|▌         | 2/33 [00:01<00:24,  1.24it/s]

...

 94%|█████████▍| 31/33 [00:23<00:01,  1.44it/s]

processing sentence {'chapter': 4, 'sentences': '今天會議的議題是討論收購滬星公司的可行性。'}


 97%|█████████▋| 32/33 [00:23<00:00,  1.50it/s]

processing sentence {'chapter': 4, 'sentences': '我們下週再開會討論吧。'}


100%|██████████| 33/33 [00:24<00:00,  1.35it/s]


In [24]:
# Convert and write JSON object to file
with open(f"../data/output/sentences/sentences.json", "w") as outfile: 
    json.dump(sentences_dict, outfile)
# print(f"successfully output .json for {chinese_word}")
# #break

In [25]:
sentences_dict[1]

{'chapter': 1,
 'sentences': '我的名字叫張宏岳，宏是宏大的宏，岳是岳飛的岳。',
 'pinyin': 'Wǒ de míngzì jiào zhānghóngyuè, hóng shì hóngdà de hóng, yuè shì yuèfēi de yuè.',
 'translation': 'My name is Zhang Hongyue, Hong means Hongda Hong, and Yue means Yue Fei.',
 'pronounciation_audio_bytes': '...'}

# Pydantic Consumption for App

In [3]:
with open("../data/output/sentences/sentences.json") as json_file:
        sentences_dict = json.load(json_file)

In [4]:
len(sentences_dict)

33

In [5]:
class SentencesEntry(BaseModel):
    chapter: int
    sentences:str
    pinyin: str
    translation: str
    pronounciation_audio_bytes: str

# Declaring the full list type that holds multiple word entries.
class SentDictionaryEntries(BaseModel):
    entries: List[SentencesEntry]

In [10]:
# Deserializing the JSON data into Pydantic models
chinese_sentences = SentDictionaryEntries(entries=[SentencesEntry(**entry) for entry in sentences_dict]).entries

In [ ]:
chinese_sentences[0]

In [12]:
chapter_to_query = 1
chapter_1 = list(filter(lambda sentence: sentence.chapter == 1, chinese_sentences))


In [32]:
IPython.display.Audio(base64.b64decode(chinese_sentences.entries[0].pronounciation_audio_bytes.encode('utf-8')))

# Consolidating json

In [40]:
compiled_keywords = []
for json_word in tqdm(glob("../data/output/keywords/**.json")):
    # Opening JSON file
    with open(json_word) as json_file:
        compiled_keywords.append(json.load(json_file))


100%|██████████| 209/209 [00:03<00:00, 67.41it/s]


In [89]:
# Convert and write JSON object to file
with open(f"../data/output_compiled/keywords/keywords.json", "w") as outfile: 
    json.dump(loaded_keywords, outfile)